<a href="https://colab.research.google.com/github/gabeno/ML-Course/blob/main/Is_it_a_bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Is it a bird?

A deep learning introduction example to check if a given photo is that of a bird.

#### Step 1: Download images of birds and non-birds

In [ ]:
!pip install -Uqq fastai duckduckgo_search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import L

def search_images(term, max_images=200):
  return L(ddg_images(term, max_results=max_images)).itemgot("image")

In [ ]:
urls = search_images("bird photos", max_images=1)
urls

download a sample and take a look at it

In [ ]:
from fastdownload import download_url
from fastai.vision.all import Image

dest = "bird.jpg"
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256, 256)

sample a forest photo

In [ ]:
dest = "forest.jpg"
download_url(search_images("forest photos", max_images=1)[0], dest, show_progress=False)
Image.open(dest).to_thumb(256, 256)

Searches seem reasonable. Lets grab a bigger sample size.

In [ ]:
from fastai.vision.utils import download_images, resize_images, verify_images, get_image_files
from pathlib import Path
from time import sleep

searches = "forest", "bird"
path = Path("bird_or_not")

for term in searches:
  dest = (path/term)
  dest.mkdir(exist_ok=True, parents=True)
  download_images(dest, urls=search_images(f"{term} photo"))
  sleep(10)
  download_images(dest, urls=search_images(f"{term} sun photo"))
  sleep(10)
  download_images(dest, urls=search_images(f"{term} shade photo"))
  sleep(10)
  resize_images(path/term, max_size=400, dest=path/term)

#### Step 2: Train our model

Some photos might not download correctly which would cause our model to fail so we remove them.

In [ ]:
from fastai.vision.utils import verify_images, get_image_files

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Use `DataLoaders` to create a training ad validation set

In [ ]:
from fastai.data.transforms import RandomSplitter, parent_label
from fastai.data.block import DataBlock, CategoryBlock
from fastai.vision.data import ImageBlock # @todo: harmonize this import
from fastai.vision.augment import Resize

data_loader = DataBlock(
    # inputs to our model are images, and the outputs are categories
    blocks=(ImageBlock, CategoryBlock),
    # find all the inputs to our model
    get_items=get_image_files,
    # split the data into training and validation sets randomly,
    # use 20% of the data for validation
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    # labels is the name of the parent folder for each file
    get_y=parent_label,
    # before training, resize each image to 192x192 pixels by squishing it (not cropping)
    item_tfms=[Resize(192, method="squish")]
).dataloaders(path)

data_loader.show_batch(max_n=6)

In [ ]:
from torchvision.models.resnet import resnet18
from fastai.vision.learner import vision_learner
from fastai.metrics import error_rate

# what neural network to use: vision_learner
# what architecture to use: resnet18 (with 18 layers)
# what metric to use: error_rate (what % of images are classified incorrectly)
learn = vision_learner(data_loader, resnet18, metrics=error_rate)
# starting witha model someone else has trained using some other dataset and
# adjusting weights a little bit so that the model learns to recognize your
# particular dataset
learn.fine_tune(5)

#### Step 3: Use our model

In [ ]:
from fastai.vision.core import PILImage


is_bird, _, probs = learn.predict(PILImage.create("bird.jpg"))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")